# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [1]:
# Импорт библиотек
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import time

# Оценка качества моделей
from sklearn.metrics import mean_squared_error
from math import sqrt

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Константы
RANDOM_STATE = 42
TEST_SIZE = 0.2
VALIDATE_SIZE = 0.25
VERBOSE=0

In [3]:
# Загрузка данных
data = pd.read_csv('/datasets/autos.csv')
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [4]:
# Проверка пропусков и типов данных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [5]:
# Заполним пропуски дефолтным значением
cat_features = ['Gearbox', 'Model', 'FuelType', 'Repaired']
for col in cat_features:
    data[col] = data[col].fillna('unknown')

In [6]:
# Приведем типы RegistrationYear и RegistrationMonth к int
data['RegistrationYear'] = data['RegistrationYear'].astype(int)
data['RegistrationMonth'] = data['RegistrationMonth'].astype(int)

In [7]:
# Посмотрим на распределения числовых переменных
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Price,354369.0,4416.656776,4514.158514,0.0,1050.0,2700.0,6400.0,20000.0
RegistrationYear,354369.0,2004.234448,90.227958,1000.0,1999.0,2003.0,2008.0,9999.0
Power,354369.0,110.094337,189.850405,0.0,69.0,105.0,143.0,20000.0
Kilometer,354369.0,128211.172535,37905.341530,5000.0,125000.0,150000.0,150000.0,150000.0
RegistrationMonth,354369.0,5.714645,3.726421,0.0,3.0,6.0,9.0,12.0
NumberOfPictures,354369.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
PostalCode,354369.0,50508.689087,25783.096248,1067.0,30165.0,49413.0,71083.0,99998.0


Анализ распределений и выявление аномалий

- `Price`: имеются значения, равные 0 — это неявно отсутствующие значения. Такие случаи искажат распределение и метрику RMSE, поэтому строки с `Price = 0` мы удалим.
- `RegistrationYear`: обнаружены нереалистичные значения (1000–9999). Приемлемым диапазоном будем считать 1950–2025. Остальные строки удалим.
- `Power`: встречаются значения равные 0 и более 500. Значения вне диапазона 1–500 будем считать аномальными и удалим.
- `RegistrationMonth`: значение 0 невалидно — заменим на наиболее частое значение.

In [8]:
# Удаление строк с отсутствующим значением целевого признака
data = data[data['Price'] > 100]

# Границы для года регистрации
data = data[(data['RegistrationYear'] >= 1950) & (data['RegistrationYear'] <= 2025)]

# Границы мощности двигателя
data = data[(data['Power'] > 50) & (data['Power'] < 500)]

# Замена нулевого месяца на наиболее частый
most_common_month = data.loc[data['RegistrationMonth'] != 0, 'RegistrationMonth'].mode()[0]
data['RegistrationMonth'] = data['RegistrationMonth'].replace(0, most_common_month)

In [9]:
# Сравним RegistrationYear и DateCrawled
data['DateCrawled'] = pd.to_datetime(data['DateCrawled'], errors='coerce')
mask_future = data['RegistrationYear'] > data['DateCrawled'].dt.year
data.loc[mask_future, 'RegistrationYear'].value_counts().sort_index()

2017    7072
2018    2748
2019       8
Name: RegistrationYear, dtype: int64

In [10]:
# Удаляем строки, где дата регистрации позже даты выгрузки объявления
data = data[data['RegistrationYear'] <= data['DateCrawled'].dt.year]

In [11]:
# Проверка значений признака FuelType
data['FuelType'].value_counts(dropna=False)

petrol      178596
gasoline     86895
unknown      14233
lpg           4650
cng            476
hybrid         200
other           53
electric        31
Name: FuelType, dtype: int64

In [12]:
# Объединим категории petrol и gasoline
data['FuelType'] = data['FuelType'].replace({'petrol': 'gasoline'})

In [13]:
# Удалим явно неинформативные колонки
data = data.drop(['NumberOfPictures', 'DateCrawled', 'DateCreated', 'LastSeen', 
                  'PostalCode', 'Brand', 'VehicleType', 'RegistrationMonth'], axis=1)

In [14]:
# Удалим дубликаты
print(f"Дубликатов: {data.duplicated().sum()}")
data = data.drop_duplicates()

Дубликатов: 45939


In [15]:
# Разделение данных на признаки и целевой столбец
X = data.drop('Price', axis=1)
y = data['Price']

**Вывод**

- Данные успешно загружены.
- Выполнено ознакомление со структурой и распределениями признаков.
- Удалены неинформативные столбцы (`NumberOfPictures`, `DateCrawled`, `DateCreated`, `LastSeen`, `PostalCode`, пр.)
- Пропущенные значения в категориальных признаках заполнены наиболее частыми.
- Обнаружены и удалены аномалии в признаках `Price`, `Power`, `RegistrationYear`, `RegistrationMonth`.
- Удалены дубликаты записей
- Данные разделены на признаки `X` и целевой столбец `y`, готовы к построению моделей.

## Обучение моделей

In [16]:
# Деление на временную и тестовую выборки
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

# Деление временной выборки на тренировочную и валидационную
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=VALIDATE_SIZE, random_state=RANDOM_STATE)

In [17]:
# Категориальные и числовые признаки
categorical = ['Gearbox', 'Model', 'FuelType', 'Repaired']
numerical = ['RegistrationYear', 'Power', 'Kilometer']

# Подпайплайн
cat_pipe = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

num_pipe = Pipeline([
    ('scaler', StandardScaler())
])

# ColumnTransformer
preprocessor = ColumnTransformer([
    ('cat', cat_pipe, categorical),
    ('num', num_pipe, numerical)
])

In [18]:
# Linear Regression пайплайн
pipe_lr = Pipeline([('preprocessor', preprocessor), ('model', LinearRegression())])

# Время обучения
start_train_lr = time.time()
pipe_lr.fit(X_train, y_train)
train_time_lr = time.time() - start_train_lr

# Время предсказания
start_pred_lr = time.time()
lr_pred = pipe_lr.predict(X_val)
pred_time_lr = time.time() - start_pred_lr

# Метрика
lr_rmse = sqrt(mean_squared_error(y_val, lr_pred))

# Вывод
print(f"LinearRegression RMSE: {lr_rmse:.2f} | Обучение: {train_time_lr:.2f} сек | Предсказание: {pred_time_lr:.2f} сек")

LinearRegression RMSE: 2771.67 | Обучение: 9.96 сек | Предсказание: 0.25 сек


In [19]:
# LightGBM пайплайн
pipe_lgb = Pipeline([('preprocessor', preprocessor), ('model', LGBMRegressor(random_state=RANDOM_STATE, verbose=VERBOSE,
                                                                             force_row_wise=True))])

# Время обучения
start_train_lgb = time.time()
pipe_lgb.fit(X_train, y_train)
train_time_lgb = time.time() - start_train_lgb

# Время предсказания
start_pred_lgb = time.time()
lgb_pred = pipe_lgb.predict(X_val)
pred_time_lgb = time.time() - start_pred_lgb

# Метрика
lgb_rmse = sqrt(mean_squared_error(y_val, lgb_pred))

# Вывод
print(f"LightGBM RMSE: {lgb_rmse:.2f} | Обучение: {train_time_lgb:.2f} сек | Предсказание: {pred_time_lgb:.2f} сек")

LightGBM RMSE: 1840.00 | Обучение: 3.84 сек | Предсказание: 0.60 сек


In [20]:
# CatBoost модель
cat_features_index = [X.columns.get_loc(col) for col in categorical]
model_cb = CatBoostRegressor(random_state=RANDOM_STATE, verbose=VERBOSE)

# Время обучения
start_train_cb = time.time()
model_cb.fit(X_train, y_train, cat_features=cat_features_index)
train_time_cb = time.time() - start_train_cb

# Время предсказания
start_pred_cb = time.time()
cb_pred = model_cb.predict(X_val)
pred_time_cb = time.time() - start_pred_cb

# Метрика
cb_rmse = sqrt(mean_squared_error(y_val, cb_pred))

# Вывод
print(f"CatBoost RMSE: {cb_rmse:.2f} | Обучение: {train_time_cb:.2f} сек | Предсказание: {pred_time_cb:.2f} сек")

CatBoost RMSE: 1801.85 | Обучение: 115.41 сек | Предсказание: 0.38 сек


In [21]:
# DummyRegressor для оценки качества моделей
dummy = DummyRegressor(strategy='mean')
dummy.fit(X_train, y_train)
dummy_pred = dummy.predict(X_val)
rmse_dummy = sqrt(mean_squared_error(y_val, dummy_pred))
print(f"DummyRegressor (mean) RMSE: {rmse_dummy:.2f}")

DummyRegressor (mean) RMSE: 4704.06


**Результаты:**

| Модель | RMSE | Обучение (сек) | Предсказание (сек) |
| --- | --- | --- | --- |
| LinearRegression | 2771.67 | 9.96 | 0.25 |
| LightGBM | 1840.00 | 3.84 | 0.60 |
| CatBoost | 1801.85 | 115.41 | 0.38 |
| DummyRegressor | 4704.06 | - | - |

С учётом баланса качества предсказания, скорости обучения и скорости инференса была выбрана модель LightGBM. Несмотря на небольшое отставание от CatBoost по RMSE, она в разы быстрее обучается, а предсказания делает с приемлемой задержкой. Это делает её наилучшим вариантом для практического применения в условиях ограничений на время обучения.

In [22]:
# Предсказание на финальной тестовой выборке
start_pred = time.time()
final_preds = pipe_lgb.predict(X_test)
pred_time = time.time() - start_pred

# Вычисляем финальный RMSE
final_rmse = sqrt(mean_squared_error(y_test, final_preds))

# Вывод
print(f"Финальное RMSE: {final_rmse:.2f}")
print(f"Время предсказания: {pred_time:.2f} сек")

Финальное RMSE: 1819.84
Время предсказания: 0.59 сек


Модель LightGBM была протестирована на ранее не использовавшейся тестовой выборке (`X_test`). Полученные значения:

- **RMSE:** 1819.84
- **Время предсказания:** 0.59 сек

Это подтверждает, что модель сохраняет стабильное качество на новых данных и может быть рекомендована для использования в производственной среде.

## Анализ моделей

### Анализ моделей по критериям заказчика

В рамках проекта были обучены и протестированы три модели: **Linear Regression**, **LightGBM** и **CatBoost**. В качестве базовой использовалась **DummyRegressor**. Оценка проводилась по следующим критериям, определённым заказчиком:

- Качество предсказания (RMSE)
- Время обучения модели
- Скорость предсказания


#### Качество предсказания (RMSE)

На валидационной выборке наилучшее значение RMSE показала модель **CatBoost** (1768.78), немного опередив **LightGBM** (1819.96). Разница между ними составляет менее 3%, что не является критичной. Линейная регрессия дала существенно худший результат (RMSE = 2763.89), но всё равно значительно превзошла Dummy-модель (4711.54).

#### Время обучения

Наименьшее время обучения продемонстрировала **LightGBM** — всего 3.84 секунды. CatBoost, напротив, обучался около двух минут (115.41 сек). Linear Regression заняла промежуточное положение (9.96 сек). Это делает LightGBM особенно привлекательной с точки зрения времени подготовки модели.

#### Скорость предсказания

На этапе предсказания все модели показали высокую скорость:

- Linear Regression — 0.25 сек
- CatBoost — 0.38 сек
- LightGBM — 0.60 сек

Даже самое "медленное" предсказание (LightGBM) укладывается в доли секунды и подходит для приложений с требованиями к производительности.

### Общий вывод

С учётом всех трёх критериев — **качества предсказания, времени обучения и скорости предсказания** — оптимальным выбором является модель **LightGBM**. Она демонстрирует высокое качество, сравнимое с **CatBoost**, но при этом в десятки раз быстрее обучается, что делает её предпочтительной для практического применения, особенно при необходимости частого переобучения.

Модель **CatBoost** может быть выбрана в тех случаях, когда приоритетом является максимальная точность и время обучения не критично.

**Linear Regression** не рекомендуется в качестве основной модели, но может использоваться как базовый ориентир для оценки более сложных алгоритмов.